In [ ]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import scipy.stats
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
def summarize_kendalls(ms, groupby):
    return smooth.analysis.summarize_kendalls(
        ms,
        groupby=groupby,
        x_col="dataset.samples_train",
        y_cols=smooth.analysis.get_measure_names(),
        get_pvalues=False,
    ).describe().round(2).loc[["mean", "std"]].T

# Increasing training set size

## 1-D, NNs

In [ ]:
ms = smooth.analysis.load_measures(
    "./0519_gp_1d/measures.feather",
    kind_cols=[
        ("dataset.seed", "seed"),
        ("dataset.samples_train", "samples"),
    ],
    remove_unconverged=True,
)

summarize_kendalls(ms, groupby="dataset.seed")

## 1-D, polynomials

In [ ]:
ms = smooth.analysis.load_measures(
    "./0519_gp_1d/measures_polynomials.feather",
    kind_cols=[
        ("dataset.seed", "seed"),
        ("dataset.samples_train", "samples"),
    ],
    remove_unconverged=False,
)

summarize_kendalls(ms, groupby="dataset.seed")

## n-D, init scale 0.01

In [ ]:
pd.read_feather("./0519_binary_increasing/measures.feather").iloc[0]["model.init_scale"]

In [ ]:
ms = smooth.analysis.load_measures(
      "./0519_binary_increasing/measures.feather",
    kind_cols=[("dataset.samples_train", "samples")],
    remove_unconverged=True,
)

summarize_kendalls(ms, groupby="dataset.name")

## n-D, init scale 1.0

In [ ]:
pd.read_feather("./0528_binary_increasing_is_1/measures.feather").iloc[0]["model.init_scale"]

In [ ]:
ms = smooth.analysis.load_measures(
      "./0528_binary_increasing_is_1/measures.feather",
    kind_cols=[("dataset.samples_train", "samples")],
    remove_unconverged=True,
)

summarize_kendalls(ms, groupby="dataset.name")

# Explicit regularization

In [ ]:
measure_names = [
    "gradient_norm_test",
    "path_length_f_test",
    "path_length_d_test",
    "weights_product",
]

def get_ratios(
    ms: pd.DataFrame, base_mask: pd.DataFrame, normed_col: str, match_col="dataset.name"
):
    """
    Computes normalized values of a dataframe's column by dividing by the value
    in a "corresponding" row. Used e.g. when explicitly regularizing smoothness measures
    """
    ms = ms.copy()
    base = ms[base_mask]
    assert base[match_col].is_unique

    normed_col_after = normed_col + "_normalized"

    # Inefficient, but good enough
    for _, row in base.iterrows():
        cur = ms.loc[ms[match_col] == row[match_col]]
        ms.loc[ms[match_col] == row[match_col], normed_col_after] = (
            cur[normed_col] / row[normed_col]
        )

    return ms

summary = pd.DataFrame(
    index=measure_names,
    columns=["Unregularized mean", "Regularized mean", "Lower bound", "Normalized mean", "Normalized std"],
    dtype=float,
)

summary["Lower bound"] = [0., 0., 1., 0.]

def update_summary(measures, measure_name, reg_coef_col_name, baseline=0):
    if baseline == 0:
        measure_name_normalized = measure_name + "_normalized"
    else:
        measure_name_normalized = measure_name + "_baselined_normalized"
    
    unreg_mean = measures[measures[reg_coef_col_name] == 0][measure_name].mean()
    summary.loc[measure_name, "Unregularized mean"] = unreg_mean

    largest_coef = max(measures[reg_coef_col_name])
    stats = measures[measures[reg_coef_col_name] == largest_coef].describe()

    summary.loc[measure_name, "Regularized mean"] = stats.loc["mean", measure_name]
    summary.loc[measure_name, "Normalized mean"] = stats.loc["mean", measure_name_normalized]
    summary.loc[measure_name, "Normalized std"] = stats.loc["std", measure_name_normalized]


## Gradient norm

In [ ]:
ms = smooth.analysis.load_measures(
      "./0410_gradient_reg/measures.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.gradient_norm_reg_coef", "gn"),
    ],
    remove_unconverged=True,
)
ms = ms.loc[ms["model.learning_rate"] == 0.01]

ms = smooth.analysis.get_ratios(ms, ms["model.gradient_norm_reg_coef"] == 0, "gradient_norm_test")
update_summary(ms, "gradient_norm_test", "model.gradient_norm_reg_coef")
summary.round(2)

## Weights product

In [ ]:
ms = smooth.analysis.load_measures(
      "./0413_weights_product_reg/measures_both.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.weights_product_reg_coef", "wp"),
    ],
    remove_unconverged=True,
)
ms = ms.loc[ms["model.learning_rate"] == 0.01]

ms = ms[ms["dataset.name"].isin(
    set(ms.loc[ms["model.weights_product_reg_coef"].round(5) == 3e-5, "dataset.name"])
)]

ms = smooth.analysis.get_ratios(ms, ms["model.weights_product_reg_coef"] == 0, "weights_product")
update_summary(ms, "weights_product", "model.weights_product_reg_coef")
summary.round(2)

## Function path length

In [ ]:
ms = smooth.analysis.load_measures(
      "./0508_path_length_f_reg/measures_both.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.path_length_f_reg_coef", "coef"),
    ],
    remove_unconverged=True,
)

ms = ms.loc[ms["actual_epochs"] < 25000]
ms = ms[ms["dataset.name"].isin(
    set(ms.loc[ms["model.path_length_f_reg_coef"].round(10) == 0.0001, "dataset.name"])
)]

ms["path_length_f_test_baselined"] = ms["path_length_f_test"] - 1
ms = get_ratios(ms, ms["model.path_length_f_reg_coef"] == 0, "path_length_f_test_baselined")

update_summary(ms, "path_length_f_test", "model.path_length_f_reg_coef", baseline=1)
summary.round(2)

## Gradient path length

In [ ]:
ms = smooth.analysis.load_measures(
      "./0508_path_length_d_reg/measures_both.feather",
    kind_cols=[
        ("model.learning_rate", "lr"),
        ("model.path_length_d_reg_coef", "coef"),
    ],
    remove_unconverged=True,
)

# Only take into account the models which converged in the regularized version.
ms = ms.loc[ms["actual_epochs"] < 25000]
ms = ms[ms["dataset.name"].isin(
    set(ms.loc[ms["model.path_length_d_reg_coef"].round(10) == 0.00001, "dataset.name"])
)]

ms = get_ratios(ms, ms["model.path_length_d_reg_coef"] == 0, "path_length_d_test")

update_summary(ms, "path_length_d_test", "model.path_length_d_reg_coef")
summary.round(2)